In [1]:
%pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00a 0:00:01


In [2]:
%pip install transformers[torch]

In [3]:
%pip install -U nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.7 MB/s eta 0:00:0000:010:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


In [4]:
import tqdm as notebook_tqdm
import ipywidgets as widgets
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
import evaluate
import numpy as np

2024-04-04 16:28:25.745530: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-04 16:28:25.745641: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-04 16:28:25.890352: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
train_words = load_dataset("wmt16", "de-en", split="train[:50000]")
eval_words = load_dataset("wmt16", "de-en", split="validation")
test_words = load_dataset("wmt16", "de-en", split="test")

# dataset = load_dataset("wmt16", 'de-en')

# train_words = dataset['train']
# train_words = train_words.select(range(50000))
# eval_words = dataset['validation']
# test_words = dataset['test']

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

Dataset wmt16 downloaded and prepared to /root/.cache/huggingface/datasets/wmt16/de-en/1.0.0/9e0038fe4cc117bd474d2774032cc133e355146ed0a47021b2040ca9db4645c0. Subsequent calls will reuse this data.


In [6]:
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [7]:
source_lang = "en"
target_lang = "de"
prefix = "translate English to German: "

def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [8]:
tokenized_train = train_words.map(preprocess_function, batched=True)
tokenized_eval= eval_words.map(preprocess_function, batched=True)
tokenized_test= test_words.map(preprocess_function, batched=True)

  0%|          | 0/50 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [9]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [10]:
blue = evaluate.load("bleu")
meteor = evaluate.load("meteor")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
                          
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result_bleu = blue.compute(predictions=decoded_preds, references=decoded_labels)
    result_meteor = meteor.compute(predictions=decoded_preds, references=decoded_labels)

    result = {"bleu1": result_bleu['precisions'][0], "bleu2": result_bleu['precisions'][1], 
              "blue3": result_bleu['precisions'][2], "bleu4": result_bleu['precisions'][3],
              "meteor":result_meteor['meteor']}
    return result


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [11]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
# print(model)

for i, param in enumerate(model.parameters()):
    if i < 35:
        param.requires_grad=False
#     print(i, param.requires_grad)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [12]:
print( model.encoder.block[0].layer[1])

T5LayerFF(
  (DenseReluDense): T5DenseActDense(
    (wi): Linear(in_features=512, out_features=2048, bias=False)
    (wo): Linear(in_features=2048, out_features=512, bias=False)
    (dropout): Dropout(p=0.1, inplace=False)
    (act): ReLU()
  )
  (layer_norm): T5LayerNorm()
  (dropout): Dropout(p=0.1, inplace=False)
)


In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir="translation",
    evaluation_strategy="epoch",
    learning_rate=1e-3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
#     weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    logging_strategy='epoch'
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Bleu1,Bleu2,Blue3,Bleu4,Meteor
1,1.333200,1.331585,0.578921,0.315539,0.193383,0.124874,0.352113
2,1.259300,1.344098,0.577012,0.313669,0.194358,0.127013,0.346448
3,1.209800,1.358684,0.573007,0.308932,0.189028,0.122727,0.343313
4,1.168800,1.373154,0.571660,0.307054,0.188464,0.120761,0.340303
5,1.136200,1.380759,0.571646,0.308344,0.188827,0.121137,0.342224
6,1.107000,1.388018,0.572566,0.309291,0.191042,0.123496,0.340584
7,1.081300,1.393589,0.571263,0.309517,0.190765,0.122547,0.340227
8,1.059400,1.400488,0.567553,0.304073,0.186077,0.118817,0.338389
9,1.042200,1.404969,0.569834,0.306211,0.188539,0.121024,0.339053
10,1.028200,1.408843,0.569687,0.306061,0.188680,0.121306,0.340202


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=7820, training_loss=1.1425468737511988, metrics={'train_runtime': 3854.4533, 'train_samples_per_second': 129.72, 'train_steps_per_second': 2.029, 'total_flos': 1.3353410024177664e+16, 'train_loss': 1.1425468737511988, 'epoch': 10.0})

In [14]:
trainer.save_model("translation")

In [15]:
loaded_model = AutoModelForSeq2SeqLM.from_pretrained('translation/checkpoint-7500')

In [16]:
test_trainer = Seq2SeqTrainer(
    model=loaded_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [17]:
trainer.predict(tokenized_test)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


PredictionOutput(predictions=array([[    0,  4534, 17119, ...,     0,     0,     0],
       [    0,   316, 21583, ...,     5,     1,     0],
       [    0,   316,  5877, ...,    67,    20, 19614],
       ...,
       [    0,   316,  2415, ...,  1289,   346,     6],
       [    0,  1318, 13942, ...,  2415,  5007,     7],
       [    0,  3139,  9237, ...,     6,  1199,    10]]), label_ids=array([[ 4534,     3,    15, ...,  -100,  -100,  -100],
       [  644,     3, 20909, ...,  -100,  -100,  -100],
       [  316,  5877,    29, ...,  -100,  -100,  -100],
       ...,
       [  316,  2974,  1847, ...,  -100,  -100,  -100],
       [ 1318, 13942,    35, ...,  -100,  -100,  -100],
       [ 3139,  9237,     3, ...,  -100,  -100,  -100]]), metrics={'test_loss': 1.3142482042312622, 'test_bleu1': 0.5931583603656738, 'test_bleu2': 0.33337646792403997, 'test_blue3': 0.2072585267985096, 'test_bleu4': 0.1348075611028615, 'test_meteor': 0.3601828310792968, 'test_runtime': 26.002, 'test_samples_per_secon